# 3rd Place - Using Only Rules Achieves LB 0.590!
- https://www.kaggle.com/competitions/otto-recommender-system/discussion/383013

- https://github.com/cdeotte/Kaggle-OTTO-Comp

5/6th place Solution: 
- https://www.kaggle.com/competitions/otto-recommender-system/discussion/382802

- Candiates Genreration(Numba) -> Feature Creation (Numba , Polars) -> Ranking Model (Lightgbm) -> Inference (Treelite)

### **Triển khai phương pháp dựa trên "Candidate Rerank" và Covisit Matrices**

---

#### **1. Phân tích giải pháp**
Phương pháp "Candidate Rerank" sử dụng:
- **Candidate Generation**: Chọn ra danh sách các sản phẩm ứng viên dựa trên hành vi trước đó của người dùng trong session.
- **Covisit Matrices**: Tính toán các mối quan hệ đồng ghé thăm (co-visit) giữa các sản phẩm dựa trên lịch sử hành vi.
- **Reranker Model**: Một mô hình học máy như **Gradient Boosted Trees (GBT)** để xếp hạng lại các sản phẩm ứng viên dựa trên các đặc trưng.

---

#### **2. Quy trình triển khai**

##### **A. Chuẩn bị dữ liệu**
1. **Session Splitting**:
   - Phân chia dataset thành training và validation dựa trên thời gian:
     ```python
     train = df[df['timestamp'] < split_time]
     valid = df[df['timestamp'] >= split_time]
     ```

2. **Xây dựng covisit matrices**:
   - Tạo ma trận **covisit** cho các loại hành vi như click, add-to-cart, order:
     ```python
     covisit_clicks = train.groupby(['aid_x', 'aid_y'])['count'].sum().reset_index()
     covisit_clicks = covisit_clicks.sort_values(by='count', ascending=False)
     ```

3. **Candidate Generation**:
   - Lấy danh sách sản phẩm ứng viên:
     ```python
     top_aids = [aid for aid, count in aids_counter.most_common(50)]
     ```

##### **B. Feature Engineering**
1. **Tạo các đặc trưng interaction**:
   - Sử dụng covisit matrices:
     ```python
     def extract_features(aid, covisit_matrix):
         features = covisit_matrix.loc[covisit_matrix['aid_x'] == aid].head(50)
         return features['count'].tolist()
     ```

2. **Đặc trưng người dùng và sản phẩm**:
   - Tổng số lần click, add-to-cart, order cho từng sản phẩm.
   - Khoảng cách thời gian giữa các sự kiện trong session.

##### **C. Huấn luyện mô hình reranker**
1. **Xây dựng dataset cho mô hình reranker**:
   - Gán nhãn (label) cho sản phẩm đã được mua hoặc thêm vào giỏ hàng trong session:
     ```python
     df_rerank['label'] = (df_rerank['aid'] == df_rerank['target_aid']).astype(int)
     ```

2. **Gradient Boosted Trees (GBT)**:
   - Sử dụng XGBoost hoặc LightGBM:
     ```python
     import lightgbm as lgb

     lgb_model = lgb.LGBMRanker()
     lgb_model.fit(X_train, y_train, group=g_train)
     ```

---

##### **D. Triển khai và dự đoán**
1. **Tạo danh sách ứng viên cho mỗi session**:
   - Kết hợp covisit matrices và các sản phẩm phổ biến:
     ```python
     candidates = generate_candidates(session_id, covisit_matrices)
     ```

2. **Dự đoán xếp hạng**:
   - Sử dụng mô hình reranker để xếp hạng lại:
     ```python
     predictions = lgb_model.predict(X_candidates)
     ranked_candidates = rank_candidates(predictions, candidates)
     ```

---

#### **3. Lưu ý khi triển khai**
- **Hiệu suất**:
  - Covisit matrices có thể rất lớn -> nên dùng thư viện GPU như **RAPIDS cuDF** để tăng tốc.
  - Áp dụng batching và lưu các ma trận ra file để giảm tải bộ nhớ.

- **Feature Scaling**:
  - Chuẩn hóa các đặc trưng để tránh vấn đề cân bằng trong mô hình.

- **Tối ưu hóa GBT**:
  - Dùng **bayesian optimization** để tìm hyperparameter tốt nhất:
    ```python
    from skopt import BayesSearchCV

    opt = BayesSearchCV(estimator=lgb.LGBMRanker(),
                        search_spaces={'learning_rate': (0.01, 0.1),
                                       'num_leaves': (31, 128)},
                        cv=3)
    opt.fit(X_train, y_train)
    ```

- **Dự phòng cold-start**:
  - Thêm đặc trưng cho các sản phẩm mới bằng cách sử dụng thông tin phổ biến (popularity) hoặc metadata.

---

#### **4. Công cụ và tài nguyên cần thiết**
- **GPU hỗ trợ RAPIDS**: Tăng tốc tính toán các ma trận đồng ghé thăm.
- **Tài nguyên bộ nhớ**: Covisit matrices yêu cầu lưu trữ lớn, nên sử dụng lưu trữ tạm thời như **parquet**.
- **Thư viện**: 
  - **LightGBM** hoặc **XGBoost** cho reranker.
  - **CuDF** hoặc **Pandas** cho xử lý dữ liệu.

Nếu cần triển khai cụ thể hơn hoặc tối ưu hóa code, mình có thể cung cấp thêm chi tiết!